# Find Cases and Controls from Metadata

__Import dependencies__

Set up R magic

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%bash
wget https://cran.r-project.org/src/contrib/rjson_0.2.20.tar.gz

In [ ]:
%%bash
R CMD INSTALL rjson_0.2.20.tar.gz

In [ ]:
%%R
library(rjson)

In [ ]:
import json
import pandas as pd
from functions import *

__1. Load data__

In [ ]:
experiment_to_terms_f_json = './data/experiment_to_terms.json'
term_name_to_id_f = './data/term_name_to_id.json'
experiments_in_hackathon_data_f = './data/experiments_in_hackathon_data.json'
experiment_to_type_f = './data/experiment_to_type.json'
experiment_to_study_f = './data/experiment_to_study.json'
experiment_to_real_value_terms_f = './data/experiment_to_real_value_terms.json'
experiment_to_runs_f = './data/experiment_to_runs.json'

In [ ]:
%%R
metadata_file_tsv <- read.table(file = "./data/experiment_to_terms.tsv", header = FALSE, sep = "\t")

In [ ]:
with open(experiment_to_terms_f_json, 'r') as f:
    sample_to_terms = json.load(f)    
with open(term_name_to_id_f, 'r') as f:
    term_name_to_id = json.load(f)
with open(experiments_in_hackathon_data_f, 'r') as f:
    available = set(json.load(f))
with open(experiment_to_type_f, 'r') as f:
    sample_to_type = json.load(f)
with open(experiment_to_study_f, 'r') as f:
    sample_to_study = json.load(f)
with open(experiment_to_real_value_terms_f, 'r') as f:
    sample_to_real_val = json.load(f)
with open(experiment_to_runs_f, 'r') as f:
    sample_to_runs = json.load(f)

__2. Type in the term you are looking for (in place of `'glioblastoma multiforme'`)__

In [ ]:
term = 'glioblastoma multiforme' ## <-- INPUT HERE

__3. List terms below to remove from control set__  
In the example below, `'disease', 'disease of cellular proliferation'` will be removed from the controls

In [ ]:
blacklist_terms = set([
    'disease', 
    'disease of cellular proliferation'
]) ## <-- INPUT HERE

__4. Create case and controls__

In [ ]:
case, control = term_to_run(sample_to_terms, term)
ret = match_case_to_controls(term, control, case, sample_to_terms,
    sample_to_study, blacklist_terms, term_name_to_id, sample_to_type,
    filter_poor=True, filter_cell_line=True, filter_differentiated=True,
    sample_to_runs=sample_to_runs, by_run=False)
df = ret[0]
control_confound = ret[1]
case_confound = ret[2]
tissue_intersections = ret[3]

These are the tissues and cell types we could match between cases and controls:

In [ ]:
pd.Series(sorted(tissue_intersections))

Here's a sample of some of the cases:

In [ ]:
df.loc[(df['condition'] == 'case')].head() 

Here's a sample of some of the controls:

In [ ]:
df.loc[(df['condition'] == 'control')].head() 

__5. Browse other metadata terms that are associated with cases and controls__

Enter whether you want to view cases or controls. Assign the following variable to `True` to view cases or `False` to view controls:

In [ ]:
view_cases = False ## <-- INPUT HERE

Enter the tissue or cell type on which to subset your samples:

In [ ]:
term = 'blood' ## <-- INPUT HERE

if view_cases:
    condition = 'case'
else:
    condition = 'control'
view_exps = select_case_control_experiment_set(df, condition, term)
with open('./data/term-in.json', 'w') as f:
    json.dump(view_exps, f)

The following plots the proportion of metadata terms for those terms that appear in at least 10% of the samples in the current subset:

In [ ]:
%%R
source("./Metadata_plot.R")
bp

In [ ]:
%%R
source("./Metadata_table.R")
query_disease_metadata_top10_table

In [ ]:
%%R
source("./Metadata_piecharts.R")

Enter the filename for which you would like to output these cases or controls:

In [ ]:
output_file = 'cases_vs_controls.csv' ## <- OUTPUT FILE HERE

df.to_csv(output_file)